In [1]:
%%capture
!pip install youtube-dl

In [2]:
!youtube-dl -j --flat-playlist "https://www.youtube.com/c/heritageboard/playlists?view=1&sort=dd&shelf_id=0" > pl.json

In [ ]:
!cat pl.json | awk -F'"url": "' '{print $2}'|awk -F'"' '{print $1}' | while read i;do youtube-dl -j --flat-playlist $i >> pl_videos.json || echo $i >> retry;done

In [ ]:
!youtube-dl -j --flat-playlist "https://www.youtube.com/c/heritageboard/videos?view=0&sort=dd&shelf_id=0" > uploads.json

In [ ]:
def inner(cur_id):
    if cur_id in seen:
        return
    req = requests.get(f"https://www.youtube.com/watch?v={cur_id}")
    if req.status_code != 200:
        retry.append(cur_id)
    if lic in req.text:
        cc_by.append(cur_id)
    else:
        other.append(cur_id)
    seen.append(cur_id)

In [ ]:
import json
import requests
cc_by = []
other = []
retry = []
seen = []
lic = '"contents":[{"runs":[{"text":"Creative Commons Attribution licence (reuse allowed)"'
with open("pl_videos.json") as pl_videos:
    for line in pl_videos.readlines():
        line_data = json.loads(line.strip())
        inner(line_data['id'])
with open("uploads.json") as pl_videos:
    for line in pl_videos.readlines():
        line_data = json.loads(line.strip())
        inner(line_data['id'])


In [ ]:
with open('proc.json', 'w') as outfile:
    json.dump({'cc-by': cc_by, 'other': other, 'retry': retry}, outfile)